In [3]:
# -*- coding: utf-8 -*-
from bs4 import BeautifulSoup
from datetime import datetime
import requests
import pandas as pd
import re

'''''''''''''''''''''''''''''''''''''''''''''''''''''''''
< naver 뉴스 검색시 리스트 크롤링하는 프로그램 > _select사용
- 크롤링 해오는 것 : 링크,제목,신문사,날짜,내용요약본
- 날짜,내용요약본  -> 정제 작업 필요
- 리스트 -> 딕셔너리 -> df -> 엑셀로 저장
'''''''''''''''''''''''''''''''''''''''''''''''''''''''''

# 각 크롤링 결과 저장하기 위한 리스트 선언
title_text = []
link_text = []
source_text = []
date_text = []
contents_text = []
result = {}

# 엑셀로 저장하기 위한 변수
RESULT_PATH = 'C:\\Users\\user\\Desktop\\HYUNDAI'  # 결과 저장할 경로
now = datetime.now()  # 파일이름 현 시간으로 저장하기


# 날짜 정제화 함수
def date_cleansing(test):
    try:
        # 지난 뉴스
        # 머니투데이  10면1단  2020.01.01.  네이버뉴스   보내기
        pattern = '\d+.(\d+).(\d+).'  # 정규표현식

        r = re.compile(pattern)
        match = r.search(test).group(0)  # 2020.01.01.
        date_text.append(match)

    except AttributeError:
        # 최근 뉴스
        # 이데일리  1시간 전  네이버뉴스   보내기
        pattern = '\w* (\d\w*)'  # 정규표현식

        r = re.compile(pattern)
        match = r.search(test).group(1)
        # print(match)
        date_text.append(match)


# 내용 정제화 함수
def contents_cleansing(contents):
    first_cleansing_contents = re.sub('<dl>.*?</a> </div> </dd> <dd>', '',
                                      str(contents)).strip()  # 앞에 필요없는 부분 제거
    second_cleansing_contents = re.sub('<ul class="relation_lst">.*?</dd>', '',
                                       first_cleansing_contents).strip()  # 뒤에 필요없는 부분 제거 (새끼 기사)
    third_cleansing_contents = re.sub('<.+?>', '', second_cleansing_contents).strip()
    contents_text.append(third_cleansing_contents)
    # print(contents_text)


def crawler(maxpage, query, sort, s_date, e_date):
    s_from = s_date.replace(".", "")
    e_to = e_date.replace(".", "")
    page = 1
    maxpage_t = (int(maxpage) - 1) * 10 + 1  # 11= 2페이지 21=3페이지 31=4페이지  ...81=9페이지 , 91=10페이지, 101=11페이지

    while page <= maxpage_t:
        url = "https://search.naver.com/search.naver?where=news&query=" + query + "&sort=" + sort + "&ds=" + s_date + "&de=" + e_date + "&nso=so%3Ar%2Cp%3Afrom" + s_from + "to" + e_to + "%2Ca%3A&start=" + str(
            page)

        response = requests.get(url)
        html = response.text

        # 뷰티풀소프의 인자값 지정
        soup = BeautifulSoup(html, 'html.parser')

        # <a>태그에서 제목과 링크주소 추출
        atags = soup.select('._sp_each_title')
        for atag in atags:
            title_text.append(atag.text)  # 제목
            link_text.append(atag['href'])  # 링크주소

        # 신문사 추출
        source_lists = soup.select('._sp_each_source')
        for source_list in source_lists:
            source_text.append(source_list.text)  # 신문사

        # 날짜 추출
        date_lists = soup.select('.txt_inline')
        for date_list in date_lists:
            test = date_list.text
            date_cleansing(test)  # 날짜 정제 함수사용

        # 본문요약본
        contents_lists = soup.select('ul.type01 dl')
        for contents_list in contents_lists:
            # print('==='*40)
            # print(contents_list)
            contents_cleansing(contents_list)  # 본문요약 정제화

        # 모든 리스트 딕셔너리형태로 저장
        result = {"date": date_text, "title": title_text, "source": source_text, "contents": contents_text,
                  "link": link_text}
        print(page)

        df = pd.DataFrame(result)  # df로 변환
        page += 10

    # 새로 만들 파일이름 지정
    outputFileName = '%s-%s-%s  %s시 %s분 %s초 merging.xlsx' % (
    now.year, now.month, now.day, now.hour, now.minute, now.second)
    df.to_excel(RESULT_PATH + outputFileName, sheet_name='sheet1')


def main():
    info_main = input("=" * 50 + "\n" + "입력 형식에 맞게 입력해주세요." + "\n" + " 시작하시려면 Enter를 눌러주세요." + "\n" + "=" * 50)

    maxpage = input("최대 크롤링할 페이지 수 입력하시오: ")
    query = input("검색어 입력: ")
    sort = input("뉴스 검색 방식 입력(관련도순=0  최신순=1  오래된순=2): ")  # 관련도순=0  최신순=1  오래된순=2
    s_date = input("시작날짜 입력(2020.07.24):")  # 2020.01.01
    e_date = input("끝날짜 입력(2020.08.24):")  # 2020.06.15

    crawler(maxpage, query, sort, s_date, e_date)


main()

입력 형식에 맞게 입력해주세요.
 시작하시려면 Enter를 눌러주세요.
최대 크롤링할 페이지 수 입력하시오: 3000
검색어 입력: yg
뉴스 검색 방식 입력(관련도순=0  최신순=1  오래된순=2): 0
시작날짜 입력(2020.07.24):2020.06.22
끝날짜 입력(2020.08.24):2020.07.22
1
11
21
31
41
51
61
71
81
91
101
111
121
131
141
151
161
171
181
191
201
211
221
231
241
251
261
271
281
291
301
311
321
331
341
351
361
371
381
391
401
411
421
431
441
451
461
471
481
491
501
511
521
531
541
551
561
571
581
591
601
611
621
631
641
651
661
671
681
691
701
711
721
731
741
751
761
771
781
791
801
811
821
831
841
851
861
871
881
891
901
911
921
931
941
951
961
971
981
991
1001
1011
1021
1031
1041
1051
1061
1071
1081
1091
1101
1111
1121
1131
1141
1151
1161
1171
1181
1191
1201
1211
1221
1231
1241
1251
1261
1271
1281
1291
1301
1311
1321
1331
1341
1351
1361
1371
1381
1391
1401
1411
1421
1431
1441
1451
1461
1471
1481
1491
1501
1511
1521
1531
1541
1551
1561
1571
1581
1591
1601
1611
1621
1631
1641
1651
1661
1671
1681
1691
1701
1711
1721
1731
1741
1751
1761
1771
1781
1791
1801
1811
1821
1831
1841
1851
1861


15051
15061
15071
15081
15091
15101
15111
15121
15131
15141
15151
15161
15171
15181
15191
15201
15211
15221
15231
15241
15251
15261
15271
15281
15291
15301
15311
15321
15331
15341
15351
15361
15371
15381
15391
15401
15411
15421
15431
15441
15451
15461
15471
15481
15491
15501
15511
15521
15531
15541
15551
15561
15571
15581
15591
15601
15611
15621
15631
15641
15651
15661
15671
15681
15691
15701
15711
15721
15731
15741
15751
15761
15771
15781
15791
15801
15811
15821
15831
15841
15851
15861
15871
15881
15891
15901
15911
15921
15931
15941
15951
15961
15971
15981
15991
16001
16011
16021
16031
16041
16051
16061
16071
16081
16091
16101
16111
16121
16131
16141
16151
16161
16171
16181
16191
16201
16211
16221
16231
16241
16251
16261
16271
16281
16291
16301
16311
16321
16331
16341
16351
16361
16371
16381
16391
16401
16411
16421
16431
16441
16451
16461
16471
16481
16491
16501
16511
16521
16531
16541
16551
16561
16571
16581
16591
16601
16611
16621
16631
16641
16651
16661
16671
16681
16691
16701
1671

28711
28721
28731
28741
28751
28761
28771
28781
28791
28801
28811
28821
28831
28841
28851
28861
28871
28881
28891
28901
28911
28921
28931
28941
28951
28961
28971
28981
28991
29001
29011
29021
29031
29041
29051
29061
29071
29081
29091
29101
29111
29121
29131
29141
29151
29161
29171
29181
29191
29201
29211
29221
29231
29241
29251
29261
29271
29281
29291
29301
29311
29321
29331
29341
29351
29361
29371
29381
29391
29401
29411
29421
29431
29441
29451
29461
29471
29481
29491
29501
29511
29521
29531
29541
29551
29561
29571
29581
29591
29601
29611
29621
29631
29641
29651
29661
29671
29681
29691
29701
29711
29721
29731
29741
29751
29761
29771
29781
29791
29801
29811
29821
29831
29841
29851
29861
29871
29881
29891
29901
29911
29921
29931
29941
29951
29961
29971
29981
29991
